In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve

In [ ]:
heart = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
df=heart.copy()
df.head()
#columns names
df.columns



    Age : Age of the patient

    Sex : Sex of the patient

    exang: exercise induced angina (1 = yes; 0 = no)

    ca: number of major vessels (0-3)

    cp : Chest Pain type chest pain type
        Value 1: typical angina
        Value 2: atypical angina
        Value 3: non-anginal pain
        Value 4: asymptomatic

    trtbps : resting blood pressure (in mm Hg)

    chol : cholestoral in mg/dl fetched via BMI sensor

    fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

    rest_ecg : resting electrocardiographic results
        Value 0: normal
        Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria

    thalach : maximum heart rate achieved

    target : 0= less chance of heart attack 1= more chance of heart attack

n 

In [ ]:
print('SHAPE'.center(80,"*"))
print('rows:{} columns:{} '.format(df.shape[0],df.shape[1]))
print('datatypes'.center(80,"*"))
print(df.dtypes)
print('descriptions'.center(80,"*"))
print(df.describe(include="all"))
print('Info'.center(80,"*"))
print(df.info())



Analysing varibales individually(Univariate analysis)

In [ ]:
#AGE
print(df['age'].isnull().sum())

ax=sns.distplot(df['age'],
                  bins=100,
                  kde=True,
                  color='skyblue',
                  hist_kws={"linewidth": 15,'alpha':1})

bx = sns.boxplot(data=df['age'], orient="h", palette="Set2")

df['age'].value_counts(dropna=False)
#zscore
from scipy import stats
z_score=np.abs(stats.zscore(df['age']))
print(z_score>3)

In [ ]:

print(df['sex'].isnull().sum())
ax=sns.countplot(df['sex']
              )
print(df['age'].isnull().sum())

In [ ]:
for colmns in df.columns:
    print(colmns)
    print(set(df[colmns]))
    print(df[colmns].isnull().sum())

In [ ]:

for cols in df:
    if df[cols].nunique() < 10:
        plt.show()
        sns.countplot(df[cols])
        
   
    
    

In [ ]:
for cols in df:
    if df[cols].nunique() > 10:
        plt.show()
        sns.distplot(df[cols])
        
   

In [ ]:
for cols in df:
    if df[cols].nunique() > 10:
        plt.show()
        sns.boxplot(df[cols])
        

In [ ]:

for cols in df:
    if df[cols].nunique() > 10:
        
        z_score=np.abs(stats.zscore(df[cols]))
        print(z_score>3)

In [ ]:
sns.pairplot(df,palette='Accent',hue='output')

In [ ]:
#pearson ion using 
df.corr()
plt.show()
f,ax=plt.subplots(figsize = (14,14))
sns.heatmap(df.corr(),annot= True,fmt = ".2f",annot_kws={"size": 12} ,vmin = -1,ax=ax, vmax = 1, linewidth = 0.4,)

In [ ]:
#want to find distributions of each variabele using shapiro test
from scipy.stats import shapiro  
stats=[]
for cols in df.columns:
    
    stat,p=shapiro(np.array(df[cols]))
    stats.append(stat)
    print(stat)
fig = plt.figure(figsize =(10, 7))
plt.bar(df.columns,stats,width=.4)

In [ ]:
#Pre processing
#missing values correction
df.isnull().sum()

In [ ]:
#duplicate rows in tthe data'
df.duplicated().sum()
df[df.duplicated()]
df.drop_duplicates(keep='first',inplace=True)
df.duplicated().sum()

In [ ]:

def outliers_detect(col):
    ascv=[]
    df[col]
    for v in df[col]:
        ascv.append(v)
    ascv.sort()

    Q1,Q3=np.percentile(ascv,[25,75])
    IQR=Q3-Q1
    low_range=Q1-(1.5*IQR)
    high_range=Q3+(1.5*IQR)
   
    filter1=df[col]>high_range
    filter2=df[col]<low_range  
    outliers=df.where(filter1 | filter2 ,axis=0)
    return df[outliers[cols].notnull()].index.tolist()

In [ ]:
outliers={}
outliers_list=[]
for cols in df.columns:
    if df[cols].nunique() > 10 :
        outliers[cols]=outliers_detect(cols)
        outliers_list.append(outliers_detect(cols))


In [ ]:
from itertools import chain
outliers_list=set(list(chain(*outliers_list)))

In [ ]:
outliers

In [ ]:

df=df.drop(labels=outliers_list,axis=0)

In [ ]:
#scaling the values

In [ ]:
test_size = 0.3
random_state = 42
y=df['output']
X=df.drop("output",axis=1)
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=.3)

In [ ]:
X_train

In [ ]:
scaler = StandardScaler() #StandardScaler - RobustScaler
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train


In [ ]:

r=GaussianNB()
r.fit(X_train,Y_train)
result=r.predict(X_test)
y_pred=result
print(y_pred)
print(classification_report(Y_test, y_pred, target_names=['0','1']))
print(confusion_matrix(Y_test,y_pred))    
    

In [ ]:
sgd = SGDClassifier(loss='hinge',random_state=random_state)
sgd.fit(X_train,Y_train)
result=r.predict(X_test)
y_pred=result
print(y_pred)
print(classification_report(Y_test, y_pred, target_names=['0','1']))
print(confusion_matrix(Y_test,y_pred))

    

In [ ]:
lda = LinearDiscriminantAnalysis(solver='svd',n_components = 1)
sgd = SGDClassifier(loss='hinge',random_state=random_state)
sgd.fit(lda.fit_transform(X_train,Y_train),Y_train)
result=r.predict(lda.transform(X_test))
y_pred=result
print(y_pred)
print(classification_report(Y_test, y_pred, target_names=['0','1']))
print(confusion_matrix(Y_test,y_pred))

    


In [ ]:

r=GaussianNB()
r.fit(lda.fit_transform(X_train,Y_train),Y_train)
result=r.predict(lda.transform(X_test))
y_pred=result
print(y_pred)
print(classification_report(Y_test, y_pred, target_names=['0','1']))
print(confusion_matrix(Y_test,y_pred))    
    

In [ ]:
#models using grid search cv
accuracy=[]
roc_auc_scores = []
recall_scores = []
precision_scores = []
f1_scores = []

models=[RandomForestClassifier(random_state = random_state), LogisticRegression(random_state = random_state),
              KNeighborsClassifier()]

para_rf={"max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}
para_lr={"C":np.logspace(-4, 4, 20),
                    "penalty": ["l1","l2","none"]}
para_knn={"n_neighbors": np.linspace(2,20,12, dtype = int).tolist(),
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan","minkowski"],
                 "leaf_size": [1,3,5,12,30]}

model_para=[para_rf,para_lr,para_knn]

for model in range(len(models)):
    
    clf = GridSearchCV(models[model],param_grid=model_para[model],cv = StratifiedKFold(n_splits = 10),
                        scoring = "accuracy",n_jobs = -1,verbose = 2)
    clf.fit(X_train,Y_train)
    print(clf.best_estimator_)
    accuracy.append(accuracy_score(clf.predict(X_test),Y_test))
    roc_auc_scores.append(roc_auc_score(Y_test, clf.predict_proba(X_test)[:,1]))
    recall_scores.append(recall_score(Y_test, clf.predict(X_test)))
    precision_scores.append(precision_score(Y_test, clf.predict(X_test), average='weighted'))
    f1_scores.append(f1_score(Y_test, clf.predict(X_test), average='weighted'))
  
    
   

In [ ]:
cv_results = pd.DataFrame({"Accuracy":accuracy,
                           
                           "ROC AUC":roc_auc_scores,
                           "Recall": recall_scores,
                           "Precision": precision_scores,
                           "F1-Score":f1_scores,
                           "Models": models})


In [ ]:
cv_results.index = cv_results["Models"]
cv_results  = cv_results.drop(["Models"], axis = 1)

f,ax = plt.subplots(figsize=(14,10))
sns.heatmap(cv_results, annot=True,cmap = "Purples",
            fmt= '.3f',ax=ax,linewidths = 5,
            cbar = False,annot_kws={"size": 18})

plt.xticks(size = 16)
plt.yticks(size = 16, rotation = 0)
plt.title("Grid Search Results", size = 16)
plt.show()  